In [1]:
# The code was removed by Watson Studio for sharing.

# **IBM Data Science** | Coursera Capstone Project - Course 9 - Week 4 and 5

### Business Problem

A pizza chain wants to open outlets in the Toronto neighborhoods, and they are looking for guidance on which neighborhoods might be suitable locations for their restaurants. They have a fixed budget that will allow them to about 10 outlets in the first year, and their management must be able to show traction in the first year as to be able to expand the business in the subsequent years. In their experience in other markets and cities, they have learned that certain locations tend to perform better than others, and locations performing well had some things in common, like they had other comfort food location making it a destination for young crowd. The initial business plan is to follow similar footsteps as in other markets, hence management wants to conduct a study to look for such similarities in the Toronto neighborhoods that could predict their success for new pizza outlets.

The management want to decide on the new location based on analysis of data on Toronto neighborhoods, the existing restaurants in those neighborhoods that are popular venues. After initial review of the Toronto neighborhoods, it was decided that they would segregate the market by the postal codes for the study and target to open one restaurant in one postal code at the maximum. The management needs information on which postal codes would likely be successful location or new pizza outlets. The management needs the popular venues in these postal codes and other parameters of the postal codes to be analyzed to determine the locations that could be successful targets for new pizza locations. 

### Data to be used for this analysis

The following datasets to be utilized for this analysis. The Toronto neighborhood information from Wikipedia will be the source for basic neighborhood information, latitudes, longitudes. The location and venue data from Foursquare will be utilized to get information related to venues in the neighborhoods, the popular venues in the neighborhoods.  

- The Toronto neighborhood data available publicly on Wikipedia (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)
- Location data for Toronto from Foursquare (www.foursquare.com)
- Toronto Geo-Spatial data from (http://cocl.us/Geospatial_data)

In [2]:
import pandas as pd
import numpy as np

#Read the Canadian Postal Codes and Boroughs from the Wikipedia page into a Pandas dataframe

wikiurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Read the Wikipedia html into pandas dataframe. Sections of the html will be read as separate dataframes. Result is a dataframe of dataframes
#canadawiki = pd.read_html(wikiurl)

# After inspecting the result dataframe of dataframes you will find that the first dataframe contains the postcode data
#canadawiki[0]

# The wikipedia page changed while I was in the middle of the project, hence old code had to be changed
# Hence I have uploaded a csv file with the Canada Postal Code information

my_file = projectabhijit.get_file("Postal Codes.csv")
my_file.seek(0)
canadawiki=pd.read_csv(my_file)
canadawiki.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
# separate the postcode data into a separate dataframe, then check the result

canadadf = pd.DataFrame(canadawiki)
canadadf.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
canadadf.shape

(302, 3)

In [5]:
# Check the number of rows that have Boroughs as Not Assigned, the number is 77, these rows need to be dropped. 
# That was my interpretation of the part of the question "Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned."

canadadf['Borough'].value_counts()

Not assigned                             77
Etobicoke                                44
North York                               38
Scarborough                              38
Downtown Toronto                         37
Central Toronto                          16
West Toronto                             13
Etobicoke Northwest                       9
East Toronto                              8
York                                      8
East York                                 6
West                                      2
Canada Post Gateway Processing Centre     1
Central                                   1
Mississauga                               1
East                                      1
Northwest                                 1
South                                     1
Name: Borough, dtype: int64

In [6]:
# Drop any rows that have NaN values, there are no NaN values

canadadf.dropna()
canadadf.shape

(302, 3)

In [7]:
# Drop the rows for which Borough is Not Assigned, check the size of new dataframe

canadadf = canadadf[canadadf.Borough !='Not assigned']
canadadf.shape

(225, 3)

In [8]:
# Find number of rows that have neighbourhood equals not assigned

len(canadadf[canadadf['Neighbourhood'] == 'Not assigned'])

7

In [9]:
# Replace neighbourhood with Borough if neighbourhood is not assigned

canadadf['Neighbourhood'] = np.where(canadadf['Neighbourhood']=='Not assigned', canadadf['Borough'], canadadf['Neighbourhood'])

len(canadadf[canadadf['Neighbourhood'] == 'Not assigned'])

0

In [10]:
# Group by the PostCode and Borough and concatenate the Neighborhoods, separated by commas

canadadf = canadadf.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
canadadf.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
canadadf.shape

(111, 3)

In [12]:
# Geocode postal codes for latitude and Longitude and assign to dataframe

geocodeurl = 'http://cocl.us/Geospatial_data'

canadageo = pd.read_csv(geocodeurl)
canadageo.rename(columns={'Postal Code':'Postcode'}, inplace=True)
canadageo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
canadadf['Latitude'] = 43.806686
canadadf['Longitude'] = -79.194353
    
for i in canadadf.index:
    for j in canadageo.index:
        if canadadf['Postcode'][i] == canadageo['Postcode'][j]:
            canadadf['Latitude'][i] = canadageo['Latitude'][j]
            canadadf['Longitude'][i] = canadageo['Longitude'][j]
            
    
    
#canadadf['Latitude'] = np.where(canadadf['Postcode']==canadageo['Postcode'], canadageo['Latitude'], 0)
#canadadf['Longitude'] = np.where(canadadf['Postcode']==canadageo['Postcode'], canadageo['Longitude'], 0)

canadadf.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
canadadf.shape

(111, 5)

In [16]:
# Import all the packages Nominatim, json, requests, kmeans and folium 

# library to handle JSON files
import json 

# Import requests to handle requests
import requests
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# import k-means from clustering
from sklearn.cluster import KMeans

In [17]:
# Nominatim to convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [18]:
# Folium map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 1

In [19]:
# Find the latitude and longitude of Toronto, Canada

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitudeca = location.latitude
longitudeca = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitudeca, longitudeca))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitudeca, longitudeca], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(canadadf['Latitude'], canadadf['Longitude'], canadadf['Borough'], canadadf['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


In [21]:
CLIENT_ID = 'NH304TZ3CSPAZBA4IPW3S3OXKQNFHAI3CMBFOFXWWQPM5XRH' # Foursquare ID
CLIENT_SECRET = 'ZWG1PXMXIAJ12ZXZUNMX4O5C3NHNAQN1HMKHTNGO2NELBOMM' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: NH304TZ3CSPAZBA4IPW3S3OXKQNFHAI3CMBFOFXWWQPM5XRH
CLIENT_SECRET:ZWG1PXMXIAJ12ZXZUNMX4O5C3NHNAQN1HMKHTNGO2NELBOMM


In [22]:
# Function will get the name, category, latitude and Longitude of the nearby venues in a neighbourhood
# This neighborhood values will be passed from the Canadadf dataframe which contains all neighbourhoods in Toronto
# I have pulled 50 venues for a neighborhood from foursquare

limit = 25

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    
    # This will loop through all the neighborhoods in the lists passed to the function from the canadadf dataframe
    # All operations to pull venues for a neighborhood from Foursquare and then curate the venues list is done inside the loop one neighborhood at a time
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # For each neighborhood, inside the loop we create the Foursquare request URL using the Latitude and Longitude of the neighbourhood, radius of 500 and limit 50 venues
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query=food&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request to explore venues within 500 radius of the neighbourhood this iteration is running for
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Extract venue information from the results returned by Foursquare and add it to venues_list dataframe
        # The name of the neighborhood, lat and long of the neighborhood are added as the first three fields
        # This loops through the results one venue at a time and adds the venues_list 
        # One entry in the list is created for each venue in the neighborhood, with the name, location and category from the results returned by Fourquare
        # The name of the first category is taken as the category of the venue
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    # Create nearby_venues dataframe using the venues list create above
    # Logic loops though one venue in the list at a time an then add the fields to the dataframe
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Create new dataset of neighborhoods with Boroughs that have Toronto in their name

torontodf = canadadf[canadadf['Borough'].str.contains('Toronto')]
print(torontodf.head())
print(torontodf.shape)

   Postcode       Borough                  Neighbourhood   Latitude  Longitude
43      M4E  East Toronto                    The Beaches  43.676357 -79.293031
46      M4J  East Toronto              The Danforth East  43.685347 -79.338106
48      M4K  East Toronto    The Danforth West,Riverdale  43.679557 -79.352188
49      M4L  East Toronto  India Bazaar,The Beaches West  43.668999 -79.315572
50      M4M  East Toronto                Studio District  43.659526 -79.340923
(40, 5)


In [23]:
# Create dataframe with venues near the neighborhoods of Toronto

toronto_venues = getNearbyVenues(names=torontodf['Neighbourhood'],
                                   latitudes=torontodf['Latitude'],
                                   longitudes=torontodf['Longitude']
                                  )

The Beaches
The Danforth East
The Danforth West,Riverdale
India Bazaar,The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Summerhill West,Rathnelly,South Hill,Forest Hill SE,Deer Park
Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond,Adelaide,King
Harbourfront East,Union Station,Toronto Islands
Toronto Dominion Centre,Design Exchange
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North & West
The Annex,North Midtown,Yorkville
University of Toronto,Harbord
Kensington Market,Chinatown,Grange Park
CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport
Enclave of M5E
First Canadian Place,Underground city
Christie
Dufferin,Dovercourt Village
Little Portugal,Trinity
Brockton,Parkdale Village,Exhibition Place
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede,Swansea


In [24]:
# Check the size of the venues list for Toronto

print(toronto_venues.shape)
toronto_venues.head()

(643, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place
1,The Beaches,43.676357,-79.293031,Fearless Meat,43.680337,-79.290289,Burger Joint
2,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
3,The Danforth East,43.685347,-79.338106,105 Eaton Bakery,43.684719,-79.340729,Bakery
4,The Danforth East,43.685347,-79.338106,Command Centre,43.687667,-79.339490,Greek Restaurant


In [25]:
# Lets see the count of venues by neighborhoods

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,25,25,25,25,25,25
"Brockton,Parkdale Village,Exhibition Place",15,15,15,15,15,15
Business Reply Mail Processing Centre 969 Eastern,5,5,5,5,5,5
"CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport",2,2,2,2,2,2
Central Bay Street,25,25,25,25,25,25
Christie,7,7,7,7,7,7
Church and Wellesley,25,25,25,25,25,25
"Commerce Court,Victoria Hotel",25,25,25,25,25,25
Davisville,25,25,25,25,25,25


In [26]:
# Lets find the number of unique venue categories 

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print(toronto_venues['Venue Category'].unique())

There are 70 uniques categories.
['Pizza Place' 'Burger Joint' 'Asian Restaurant' 'Bakery'
 'Greek Restaurant' 'Italian Restaurant' 'Restaurant' 'Diner' 'Café'
 'Caribbean Restaurant' 'Indian Restaurant' 'Japanese Restaurant'
 'American Restaurant' 'Sushi Restaurant' 'Fish & Chips Shop'
 'Fast Food Restaurant' 'Steakhouse' 'Burrito Place' 'Sandwich Place'
 'Middle Eastern Restaurant' 'Comfort Food Restaurant' 'Thai Restaurant'
 'Seafood Restaurant' 'Latin American Restaurant' 'Gastropub' 'Donut Shop'
 'Dim Sum Restaurant' 'Breakfast Spot' 'Chinese Restaurant'
 'Mexican Restaurant' 'Bagel Shop' 'French Restaurant'
 'New American Restaurant' 'Fried Chicken Joint' 'Vietnamese Restaurant'
 'Deli / Bodega' 'Taiwanese Restaurant' 'Theme Restaurant'
 'Ramen Restaurant' 'Ethiopian Restaurant' 'Creperie' 'Wings Joint'
 'Food Truck' 'Modern European Restaurant' 'BBQ Joint' 'Poke Place'
 'Molecular Gastronomy Restaurant' 'Vegetarian / Vegan Restaurant'
 'Bistro' 'Belgian Restaurant' 'Eastern Euro

In [46]:
# Here the unique venue categories are converted to columns using one hot encoding

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add the neighborhood column back to new dataframe so each row can be identified for a neighborhood

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
toronto_onehot['Latitude'] = toronto_venues['Neighborhood Latitude']
toronto_onehot['Longitude'] = toronto_venues['Neighborhood Longitude']

# move neighborhood column to the first column

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Longitude,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,...,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Neighborhood,Latitude
0,-79.293031,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,The Beaches,43.676357
1,-79.293031,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,The Beaches,43.676357
2,-79.293031,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,The Beaches,43.676357
3,-79.338106,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,The Danforth East,43.685347
4,-79.338106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,The Danforth East,43.685347


In [47]:
print(toronto_onehot.shape)
print(toronto_onehot.columns)

(643, 73)
Index(['Longitude', 'American Restaurant', 'Arepa Restaurant',
       'Asian Restaurant', 'BBQ Joint', 'Bagel Shop', 'Bakery',
       'Belgian Restaurant', 'Bistro', 'Brazilian Restaurant',
       'Breakfast Spot', 'Burger Joint', 'Burrito Place', 'Café',
       'Cajun / Creole Restaurant', 'Caribbean Restaurant',
       'Chinese Restaurant', 'Colombian Restaurant', 'Comfort Food Restaurant',
       'Creperie', 'Cuban Restaurant', 'Deli / Bodega', 'Dim Sum Restaurant',
       'Diner', 'Donut Shop', 'Dumpling Restaurant',
       'Eastern European Restaurant', 'Ethiopian Restaurant',
       'Falafel Restaurant', 'Fast Food Restaurant', 'Fish & Chips Shop',
       'Food', 'Food Court', 'Food Truck', 'French Restaurant',
       'Fried Chicken Joint', 'Gastropub', 'Gluten-free Restaurant',
       'Greek Restaurant', 'Indian Restaurant', 'Irish Pub',
       'Italian Restaurant', 'Japanese Restaurant', 'Korean Restaurant',
       'Latin American Restaurant', 'Mac & Cheese Joint', 'M

In [48]:
# There are multiple rows in the onehot dataframe for each neighborhood
# Here we group by the neighborhood and take the mean of the occurence of a venue category
# The result will have as many rows as the number of neighborhoods

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped

,Neighborhood,Longitude,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,...,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Latitude
0,Berczy Park,-79.373306,0.000000,0.00,0.000000,0.040000,0.040000,0.080000,0.04,0.04,...,0.040000,0.040000,0.00,0.0,0.040000,0.00,0.040000,0.000000,0.00,43.644771
1,"Brockton,Parkdale Village,Exhibition Place",-79.428191,0.000000,0.00,0.000000,0.000000,0.000000,0.133333,0.00,0.00,...,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.066667,0.00,43.636847
2,Business Reply Mail Processing Centre 969 Eastern,-79.321558,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,43.662744
3,"CN Tower,King and Spadina,Railway Lands,Harbou...",-79.394420,0.500000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.5,0.000000,0.00,0.000000,0.000000,0.00,43.628947
4,Central Bay Street,-79.387383,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.040000,0.080000,0.00,0.0,0.040000,0.00,0.040000,0.000000,0.00,43.657952
5,Christie,-79.422564,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,43.669542
6,Church and Wellesley,-79.383160,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.040000,0.080000,0.00,0.0,0.040000,0.04,0.000000,0.040000,0.04,43.665860
7,"Commerce Court,Victoria Hotel",-79.379817,0.080000,0.00,0.000000,0.000000,0.000000,0.040000,0.00,0.00,...,0.040000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,43.648199
8,Davisville,-79.388790,0.080000,0.00,0.040000,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.080000,0.00,0.0,0.080000,0.00,0.000000,0.000000,0.00,43.704324
9,Davisville North,-79.390197,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.00,0.00,...,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,43.712751


In [50]:
#Summarize or group the venue types into categories like Pizzaoutlet or North American or Comfort food
#In total there are 8 restaurant types created below

toronto_grouped_summarized= pd.DataFrame(toronto_grouped[['Neighborhood','Latitude','Longitude']])

toronto_grouped_summarized['PizzaOutlet'] = toronto_grouped['Pizza Place'] + toronto_grouped['Italian Restaurant'] + toronto_grouped['Poke Place']


toronto_grouped_summarized['North American'] = toronto_grouped['Burger Joint'  ] + toronto_grouped['Restaurant'] + toronto_grouped['Diner'] + toronto_grouped['Café'] +\
toronto_grouped['Bakery'] + toronto_grouped['American Restaurant'] + toronto_grouped['Steakhouse'] + toronto_grouped['New American Restaurant']


toronto_grouped_summarized['Comfort Food'] = toronto_grouped['Fast Food Restaurant'] + toronto_grouped['Fish & Chips Shop'] + toronto_grouped['Comfort Food Restaurant'] +\
toronto_grouped['Bagel Shop'] + toronto_grouped['Fried Chicken Joint'] + toronto_grouped['Wings Joint'] + toronto_grouped['Food Truck'] +\
toronto_grouped['BBQ Joint'] + toronto_grouped['Gastropub'] + toronto_grouped['Food Court'] + toronto_grouped['Donut Shop'] + toronto_grouped['Mac & Cheese Joint'] + toronto_grouped['Food']



toronto_grouped_summarized['Asian Cuisine'] = toronto_grouped['Noodle House'] + toronto_grouped['Asian Restaurant'] + toronto_grouped['Indian Restaurant'] +\
toronto_grouped['Japanese Restaurant'] + toronto_grouped['Sushi Restaurant'] + toronto_grouped['Thai Restaurant'] + toronto_grouped['Seafood Restaurant'] +\
toronto_grouped['Dim Sum Restaurant'] + toronto_grouped['Chinese Restaurant'] + toronto_grouped['Vietnamese Restaurant'] + toronto_grouped['Taiwanese Restaurant'] +\
toronto_grouped['Ramen Restaurant'] + toronto_grouped['Korean Restaurant'] + toronto_grouped['Malay Restaurant'] + toronto_grouped['Dumpling Restaurant']


toronto_grouped_summarized['South American Cuisine'] = toronto_grouped['Caribbean Restaurant'] + toronto_grouped['Burrito Place'] + toronto_grouped['Latin American Restaurant'] +\
toronto_grouped['Mexican Restaurant'] + toronto_grouped['Colombian Restaurant'] + toronto_grouped['Cuban Restaurant'] + toronto_grouped['Brazilian Restaurant']


toronto_grouped_summarized['Middle Eastern Cuisine'] = toronto_grouped['Ethiopian Restaurant'] + toronto_grouped['Middle Eastern Restaurant'] + toronto_grouped['Falafel Restaurant'] +\
toronto_grouped['Persian Restaurant']


toronto_grouped_summarized['European Cuisine'] = toronto_grouped['Breakfast Spot'] + toronto_grouped['French Restaurant'] + toronto_grouped['Theme Restaurant'] +\
toronto_grouped['Deli / Bodega'] + toronto_grouped['Creperie'] + toronto_grouped['Modern European Restaurant'] + toronto_grouped['Cajun / Creole Restaurant'] +\
toronto_grouped['Irish Pub'] + toronto_grouped['Greek Restaurant'] + toronto_grouped['Belgian Restaurant'] + toronto_grouped['Eastern European Restaurant'] +\
toronto_grouped['Bistro'] + toronto_grouped['Portuguese Restaurant'] + toronto_grouped['Arepa Restaurant'] + toronto_grouped['Tapas Restaurant']


toronto_grouped_summarized['Healthy Choices'] = toronto_grouped['Vegetarian / Vegan Restaurant'] + toronto_grouped['Salad Place'] + toronto_grouped['Gluten-free Restaurant'] +\
toronto_grouped['Sandwich Place'] + toronto_grouped['Molecular Gastronomy Restaurant']

toronto_grouped_summarized

,Neighborhood,Latitude,Longitude,PizzaOutlet,North American,Comfort Food,Asian Cuisine,South American Cuisine,Middle Eastern Cuisine,European Cuisine,Healthy Choices
0,Berczy Park,43.644771,-79.373306,0.040000,0.320000,0.120000,0.200000,0.000000,0.000000,0.280000,0.040000
1,"Brockton,Parkdale Village,Exhibition Place",43.636847,-79.428191,0.066667,0.466667,0.000000,0.133333,0.066667,0.000000,0.133333,0.133333
2,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0.200000,0.200000,0.400000,0.000000,0.200000,0.000000,0.000000,0.000000
3,"CN Tower,King and Spadina,Railway Lands,Harbou...",43.628947,-79.394420,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
4,Central Bay Street,43.657952,-79.387383,0.160000,0.240000,0.040000,0.360000,0.000000,0.080000,0.040000,0.080000
5,Christie,43.669542,-79.422564,0.142857,0.857143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,43.665860,-79.383160,0.080000,0.280000,0.080000,0.320000,0.080000,0.040000,0.120000,0.000000
7,"Commerce Court,Victoria Hotel",43.648199,-79.379817,0.000000,0.520000,0.120000,0.160000,0.000000,0.000000,0.080000,0.120000
8,Davisville,43.704324,-79.388790,0.160000,0.320000,0.000000,0.320000,0.000000,0.000000,0.080000,0.120000
9,Davisville North,43.712751,-79.390197,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.250000


In [53]:
#Sort the locations or neighborhoods by their rating of pizza outlets

toronto_grouped_summarized.sort_values(by='PizzaOutlet')
toronto_grouped_summarized['Recommended'] = 0

#The neighborhoods that have a pizza outlet rating of 0.5 or greater will be marked as recommended and anything less is marked as not recommended

for i in toronto_grouped_summarized.index:
    if toronto_grouped_summarized['PizzaOutlet'][i] > 0.1:
            toronto_grouped_summarized['Recommended'][i] = 1

#Drop the pizza outlet column as I dont want to use that for the classification. That may overweigh on the results

toronto_grouped_summarized.drop('PizzaOutlet', axis=1)
toronto_grouped_summarized.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Latitude,Longitude,PizzaOutlet,North American,Comfort Food,Asian Cuisine,South American Cuisine,Middle Eastern Cuisine,European Cuisine,Healthy Choices,Recommended
0,Berczy Park,43.644771,-79.373306,0.040000,0.320000,0.12,0.200000,0.000000,0.00,0.280000,0.040000,0
1,"Brockton,Parkdale Village,Exhibition Place",43.636847,-79.428191,0.066667,0.466667,0.00,0.133333,0.066667,0.00,0.133333,0.133333,0
2,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0.200000,0.200000,0.40,0.000000,0.200000,0.00,0.000000,0.000000,1
3,"CN Tower,King and Spadina,Railway Lands,Harbou...",43.628947,-79.394420,0.000000,0.500000,0.00,0.000000,0.000000,0.00,0.500000,0.000000,0
4,Central Bay Street,43.657952,-79.387383,0.160000,0.240000,0.04,0.360000,0.000000,0.08,0.040000,0.080000,1


In [54]:
#Create training data by selecting the first 8 rows and the last 8 rows
#That is - the top locations that have a higher score for pizza outlets and the bottom locations that have the lowest score for pizza outlet

toronto_grouped_test = toronto_grouped_summarized.head(8)
toronto_grouped_test.append(toronto_grouped_summarized.tail(8))

toronto_grouped_test.head(10)

,Neighborhood,Latitude,Longitude,PizzaOutlet,North American,Comfort Food,Asian Cuisine,South American Cuisine,Middle Eastern Cuisine,European Cuisine,Healthy Choices,Recommended
0,Berczy Park,43.644771,-79.373306,0.040000,0.320000,0.12,0.200000,0.000000,0.00,0.280000,0.040000,0
1,"Brockton,Parkdale Village,Exhibition Place",43.636847,-79.428191,0.066667,0.466667,0.00,0.133333,0.066667,0.00,0.133333,0.133333,0
2,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0.200000,0.200000,0.40,0.000000,0.200000,0.00,0.000000,0.000000,1
3,"CN Tower,King and Spadina,Railway Lands,Harbou...",43.628947,-79.394420,0.000000,0.500000,0.00,0.000000,0.000000,0.00,0.500000,0.000000,0
4,Central Bay Street,43.657952,-79.387383,0.160000,0.240000,0.04,0.360000,0.000000,0.08,0.040000,0.080000,1
5,Christie,43.669542,-79.422564,0.142857,0.857143,0.00,0.000000,0.000000,0.00,0.000000,0.000000,1
6,Church and Wellesley,43.665860,-79.383160,0.080000,0.280000,0.08,0.320000,0.080000,0.04,0.120000,0.000000,0
7,"Commerce Court,Victoria Hotel",43.648199,-79.379817,0.000000,0.520000,0.12,0.160000,0.000000,0.00,0.080000,0.120000,0


In [59]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

#Create the training data for the K Nearest Neighbor classification

X_train = toronto_grouped_test[['PizzaOutlet','North American', 'Comfort Food', 'Asian Cuisine', 'South American Cuisine', 'Middle Eastern Cuisine','European Cuisine','Healthy Choices']].values
y_train = toronto_grouped_test[['Recommended']].values

In [60]:
k = 8

#Train Model to classify the locations for pizza outlets using the training data

neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=8, p=2,
           weights='uniform')

In [62]:
#Create the test data which is essentially the entire list of neighborhoods
#I want to run the model on the entire neighborhood to see the ratings given to the locations by the model

X_test = toronto_grouped_summarized[['PizzaOutlet','North American', 'Comfort Food', 'Asian Cuisine', 'South American Cuisine', 'Middle Eastern Cuisine','European Cuisine','Healthy Choices']].values
yHat = neigh.predict(X_test)
yHat[0:37]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [65]:
#Add the prediction or the recommendation for pizza outlet back to the original Toronto array that contains the zipcodes
#This provides the locations and the recommendations

toronto_grouped_summarized['Recommended'] = yHat
for i in toronto_grouped_summarized.index:
    if toronto_grouped_summarized['PizzaOutlet'][i] > 0.1:
            toronto_grouped_summarized['Recommended'][i] = 1

toronto_grouped_summarized['Borough'] = ''
    
for i in toronto_grouped_summarized.index:
    for j in canadadf.index:
        if toronto_grouped_summarized['Latitude'][i] == canadadf['Latitude'][j] and toronto_grouped_summarized['Longitude'][i] == canadadf['Longitude'][j]:
            toronto_grouped_summarized['Borough'][i] = canadadf['Borough'][j]
            

toronto_grouped_summarized.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Latitude,Longitude,PizzaOutlet,North American,Comfort Food,Asian Cuisine,South American Cuisine,Middle Eastern Cuisine,European Cuisine,Healthy Choices,Recommended,Borough
0,Berczy Park,43.644771,-79.373306,0.040000,0.320000,0.12,0.200000,0.000000,0.00,0.280000,0.040000,0,
1,"Brockton,Parkdale Village,Exhibition Place",43.636847,-79.428191,0.066667,0.466667,0.00,0.133333,0.066667,0.00,0.133333,0.133333,0,
2,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0.200000,0.200000,0.40,0.000000,0.200000,0.00,0.000000,0.000000,1,East Toronto
3,"CN Tower,King and Spadina,Railway Lands,Harbou...",43.628947,-79.394420,0.000000,0.500000,0.00,0.000000,0.000000,0.00,0.500000,0.000000,0,Downtown Toronto
4,Central Bay Street,43.657952,-79.387383,0.160000,0.240000,0.04,0.360000,0.000000,0.08,0.040000,0.080000,1,


In [67]:
#Create a map of Toronto and show the locations that are recommended for pizza outlets
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitudeca, longitudeca], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_grouped_summarized['Latitude'], toronto_grouped_summarized['Longitude'], toronto_grouped_summarized['Borough'], toronto_grouped_summarized['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto